In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import r2_score
from data_utils import load_info, create_dataloaders, load_preprocessed_data

In [146]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [147]:
def l1_regularization(model, l1_lambda):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return l1_lambda * l1_norm

def train(model, train_loader, valid_loader, criterion, optimizer, epochs, patience, l1_lambda):
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.float(), targets.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets) + l1_regularization(model, l1_lambda)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in valid_loader:
                inputs, targets = inputs.float(), targets.float()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(valid_loader.dataset)
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

def test(model, test_loader):
    model.eval()
    targets_list = []
    outputs_list = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.float(), targets.float()
            outputs = model(inputs)
            
            targets_list.append(targets.numpy())
            outputs_list.append(outputs.numpy())

    outputs_list = np.concatenate(outputs_list)
    targets_list = np.concatenate(targets_list)

    r2 = r2_score(targets_list, outputs_list)
    print(f'R² score: {r2}\n\n')

In [148]:
input_data, target_data = load_preprocessed_data()
firm_info, _ = load_info()
train_loader, valid_loader, test_loader, _ = create_dataloaders(
    input_data, target_data, firm_info,
    train_date='2008-01-01', valid_date='2017-01-01', test_date='2023-01-01', batch_size=3000)

In [149]:
input_dim = input_data.shape[1] - 2
# hidden_layers_list = [[128, 64, 32], [32, 16, 8], [128, 64], [64, 32], [32, 16], [16, 8]]
hidden_layers_list = [[128, 64], [64, 32], [32, 16], [16, 8]]
output_dim = 1
learning_rates = [0.001]
epochs = 100
patience = 5
l1_lambdas = [1e-5]

In [150]:
for hidden_layers in hidden_layers_list:
    for learning_rate in learning_rates:
        for l1_lambda in l1_lambdas:
            print(f"Training with hidden_layers={hidden_layers}, learning_rate={learning_rate}, l1_lambda={l1_lambda}\n")
            model = NeuralNetwork(input_dim, hidden_layers, output_dim)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            train(model, train_loader, valid_loader, criterion, optimizer, epochs, patience, l1_lambda)
            test(model, test_loader)

Training with hidden_layers=[128, 64], learning_rate=0.001, l1_lambda=1e-05

Epoch 1/100, Train Loss: 0.034340, Val Loss: 0.022839
Epoch 2/100, Train Loss: 0.028748, Val Loss: 0.022806
Epoch 3/100, Train Loss: 0.026920, Val Loss: 0.022764
Epoch 4/100, Train Loss: 0.026129, Val Loss: 0.022782
Epoch 5/100, Train Loss: 0.025691, Val Loss: 0.022783
Epoch 6/100, Train Loss: 0.025407, Val Loss: 0.022826
Epoch 7/100, Train Loss: 0.025227, Val Loss: 0.022873
Epoch 8/100, Train Loss: 0.025085, Val Loss: 0.022942
Early stopping triggered
R² score: 0.0014224051163549412


Training with hidden_layers=[64, 32], learning_rate=0.001, l1_lambda=1e-05

Epoch 1/100, Train Loss: 0.030243, Val Loss: 0.022912
Epoch 2/100, Train Loss: 0.027763, Val Loss: 0.022832
Epoch 3/100, Train Loss: 0.026694, Val Loss: 0.022820
Epoch 4/100, Train Loss: 0.026102, Val Loss: 0.022813
Epoch 5/100, Train Loss: 0.025736, Val Loss: 0.022823
Epoch 6/100, Train Loss: 0.025485, Val Loss: 0.022832
Epoch 7/100, Train Loss: 0.02530